In [ ]:
H0: The average funding received by startups is 50 million.
 
Ha: The average funding received by startups is not 50 million.

In [607]:
# importing necessary libraries
import pyodbc     
from dotenv import dotenv_values   
import pandas as pd
import warnings 
import requests
import io
import datetime
warnings.filterwarnings('ignore')


In [608]:
# Load environment variables from .env file into a dictionary
environment_variables = dotenv_values('.env')

# Get the values for the credentials you set in the '.env' file
server = environment_variables.get("SERVER")
database = environment_variables.get("DATABASE")
username = environment_variables.get("USERNAME")
password = environment_variables.get("PASSWORD")

In [609]:
#creating a connection sring to the SQL  database
connection_string = f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password};MARS_Connection=yes;MinProtocolVersion=TLSv1.2;"

In [610]:
connection = pyodbc.connect(connection_string)

In [611]:
#querying data of year 2020
query_2020 = "SELECT*FROM dbo.LP1_startup_funding2020"
data2020 = pd.read_sql(query_2020, connection)
# print the first 5 rows
data2020.head()

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage,column10
0,Aqgromalin,2019.0,Chennai,AgriTech,Cultivating Ideas for Profit,"Prasanna Manogaran, Bharani C L",Angel investors,200000.0,None,None
1,Krayonnz,2019.0,Bangalore,EdTech,An academy-guardian-scholar centric ecosystem ...,"Saurabh Dixit, Gurudutt Upadhyay",GSF Accelerator,100000.0,Pre-seed,None
2,PadCare Labs,2018.0,Pune,Hygiene management,Converting bio-hazardous waste to harmless waste,Ajinkya Dhariya,Venture Center,NaN,Pre-seed,None
3,NCOME,2020.0,New Delhi,Escrow,Escrow-as-a-service platform,Ritesh Tiwari,"Venture Catalysts, PointOne Capital",400000.0,None,None
4,Gramophone,2016.0,Indore,AgriTech,Gramophone is an AgTech platform enabling acce...,"Ashish Rajan Singh, Harshit Gupta, Nishant Mah...","Siana Capital Management, Info Edge",340000.0,None,None


In [612]:
# showing the data type 
data2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1055 entries, 0 to 1054
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Company_Brand  1055 non-null   object 
 1   Founded        842 non-null    float64
 2   HeadQuarter    961 non-null    object 
 3   Sector         1042 non-null   object 
 4   What_it_does   1055 non-null   object 
 5   Founders       1043 non-null   object 
 6   Investor       1017 non-null   object 
 7   Amount         801 non-null    float64
 8   Stage          591 non-null    object 
 9   column10       2 non-null      object 
dtypes: float64(2), object(8)
memory usage: 82.6+ KB


In [613]:
# checking the frequency count of missing values 
data2020.apply(lambda X:sum(X.isnull())) 

Company_Brand       0
Founded           213
HeadQuarter        94
Sector             13
What_it_does        0
Founders           12
Investor           38
Amount            254
Stage             464
column10         1053
dtype: int64

In [614]:
def swap_non_numeric_with_numeric(df):
    amounts_column = 'Amount'
    stage_column = 'Stage'
    investor_column = 'Investor'

 
    # Function to find non-numeric values in 'Amounts' column
    def find_non_numeric_values():
        non_numeric_dict = {}
        for index, value in df[amounts_column].items():
            try:
                pd.to_numeric(value)
            except ValueError:
                non_numeric_dict[index] = value
        return non_numeric_dict
 
    # Function to find numeric values in 'Stage' column
    def find_numeric_values():
        numeric_dict = {}
        for index, value in df[stage_column].items():
            try:
                numeric_dict[index] = pd.to_numeric(value)
            except ValueError:
                pass  # Ignore non-numeric values
        return numeric_dict
 
    # Find non-numeric values in 'Amounts' column
    non_numeric_values = find_non_numeric_values()
 
    # Find numeric values in 'Stage' column
    numeric_values = find_numeric_values()
 
    # Swap non-numeric values in 'Amounts' with numeric values in 'Stage'
    for index, non_numeric_value in non_numeric_values.items():
        if index in numeric_values:
            df.at[index, amounts_column] = numeric_values[index]
 
    # Swap numeric values in 'Stage' with non-numeric values in 'Amounts'
    for index, numeric_value in numeric_values.items():
        if index in non_numeric_values:
            df.at[index, stage_column] = non_numeric_values[index]
 
    # Function to find numeric values in 'Investor' column
    def find_numeric_values_founders():
        numeric_dict = {}
        for index, value in df[investor_column].items():
            try:
                numeric_dict[index] = pd.to_numeric(value)
            except ValueError:
                pass  # Ignore non-numeric values
        return numeric_dict
 
    # Function to find non-numeric values in 'Amounts' column
    def find_non_numeric_values_founders():
        non_numeric_dict = {}
        for index, value in df[amounts_column].items():
            try:
                pd.to_numeric(value)
            except ValueError:
                non_numeric_dict[index] = value
        return non_numeric_dict
 
    # Find non-numeric values in 'Amounts' column
    non_numeric_values = find_non_numeric_values_founders()
 
    # Find numeric values in 'Founders' column
    numeric_values = find_numeric_values_founders()
 
    # Swap non-numeric values in 'Amounts' with numeric values in 'Founders'
    for index, non_numeric_value in non_numeric_values.items():
        if index in numeric_values:
            df.at[index, amounts_column] = numeric_values[index]
 
    # Swap numeric values in 'Founders' with non-numeric values in 'Amounts'
    for index, numeric_value in numeric_values.items():
        if index in non_numeric_values:
            df.at[index, investor_column] = non_numeric_values[index]
 
    return df
 
 
def strip_dollar_and_comma(df):
    # Assuming column names are 'Amounts' and 'Stage'
    numeric_columns = ['Investor', 'Amount', 'Stage']
 
    # Iterate through numeric columns and strip dollar sign ($) and comma (,)
    for column in numeric_columns:
        df[column] = df[column].replace('[\$]', '', regex=True)
 
    return df
 
 
def replace_non_numeric_with_zero(df):
    # Assuming column name is 'Amounts'
    amounts_column = 'Amount'
    # Replace non-numeric values in 'Amounts' column with 0
    df[amounts_column] = pd.to_numeric(df[amounts_column], errors='coerce').fillna(0).astype(int)
 
    return df


def fill_null_values(df):
    # Specify the columns to fill with 'N/A'
    columns_to_fill_na = ['HeadQuarter', 'Investor', 'Stage','Sector']
 
    # Fill null values in specified columns with 'N/A'
    df[columns_to_fill_na] = df[columns_to_fill_na].fillna('N/A')

    # Fill null values in 'Founded' with 0   
    df['Founded'] = df['Founded'].fillna(0)

    # Convert 'Founded' values to integers    
    df['Founded'] = df['Founded'].astype(int)
 
    return df


  
 
# Example usage
#data = {'Founders': ['$1,200', '$1,300,000', '', 'D', 'E', 'F', '', 'H', 'J', 'K'],
       # 'Amounts': ['A', 'Undisclosed', '$4,000', '$5,000', 'undisclosed', '$7,000,000', 'Seed', '$4,000', 'series A', '$5,000'],
       # 'Stage': ['Upsparks', 'Series C', 'series c', 'seed', 'seed', 'seed', '$2,000', 'seed', '$3,000', 'series A']}
df = pd.DataFrame(data2020)
 
# Display the DataFrame before processing
print("DataFrame before processing:")
print(df)
info = df.info()
#print("\n Info before processing")
#print(info)
 
# Call the function to replace NaN values in 'Founders' with "N/A"
df['Founders'] = df['Founders'].fillna('N/A')
 
# Call the function to strip dollar sign and comma from numeric values
df = strip_dollar_and_comma(df)
 
# Display the DataFrame after stripping dollar sign and comma
#print("\nDataFrame after stripping dollar sign and comma:")
#print(df)
 
# Call the function to swap non-numeric with numeric values
df = swap_non_numeric_with_numeric(df)
 
# Display the DataFrame after swapping Numerical values in Stage with non-numericals in amounts
#print("\nDataFrame after swapping numeric in 'Stage' with non-numeric values in 'Amount':")
#print(df)
 
# Call the function to swap numeric with non-numeric values in 'Founders'
df = swap_non_numeric_with_numeric(df)
 
# Display the DataFrame after swapping numeric with non-numeric values in 'Founders'
#print("\nDataFrame after swapping numeric in 'Founders' with non-numeric values in 'Amount':")
#print(df)
# Call the function to fill null values with 'N/A'
df = fill_null_values(df)
 # Display the DataFrame after filling null values
print("\nDataFrame after filling null values with 'N/A':")

print(df)
# Call the function to replace non-numeric values in 'Amounts' with 0
df = replace_non_numeric_with_zero(df)

# Call the function to fill null values in  'Founded' with 0
df = fill_null_values(df) 
# Display the DataFrame after filling null values
#print("\nDataFrame after filling null values with 'N/A' and 'Founded' with 0:")
#print(df)

 
 
# Display the DataFrame after replacing non-numeric values with 0
#My amounts column should have all numerics
#print("\nDataFrame after replacing non-numeric values with 0 :")
print("\nDataFrame after processing:")
cleandata_2020=df
cleandata_2020.head(10)

 


DataFrame before processing:
     Company_Brand  Founded HeadQuarter              Sector  \
0       Aqgromalin   2019.0     Chennai            AgriTech   
1         Krayonnz   2019.0   Bangalore              EdTech   
2     PadCare Labs   2018.0        Pune  Hygiene management   
3            NCOME   2020.0   New Delhi              Escrow   
4       Gramophone   2016.0      Indore            AgriTech   
...            ...      ...         ...                 ...   
1050  Leverage Edu      NaN       Delhi              Edtech   
1051         EpiFi      NaN        None             Fintech   
1052       Purplle   2012.0      Mumbai           Cosmetics   
1053        Shuttl   2015.0       Delhi           Transport   
1054         Pando   2017.0     Chennai            Logitech   

                                           What_it_does  \
0                          Cultivating Ideas for Profit   
1     An academy-guardian-scholar centric ecosystem ...   
2      Converting bio-hazardous waste

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage,column10
0,Aqgromalin,2019,Chennai,AgriTech,Cultivating Ideas for Profit,"Prasanna Manogaran, Bharani C L",Angel investors,200000,N/A,None
1,Krayonnz,2019,Bangalore,EdTech,An academy-guardian-scholar centric ecosystem ...,"Saurabh Dixit, Gurudutt Upadhyay",GSF Accelerator,100000,Pre-seed,None
2,PadCare Labs,2018,Pune,Hygiene management,Converting bio-hazardous waste to harmless waste,Ajinkya Dhariya,Venture Center,0,Pre-seed,None
3,NCOME,2020,New Delhi,Escrow,Escrow-as-a-service platform,Ritesh Tiwari,"Venture Catalysts, PointOne Capital",400000,N/A,None
4,Gramophone,2016,Indore,AgriTech,Gramophone is an AgTech platform enabling acce...,"Ashish Rajan Singh, Harshit Gupta, Nishant Mah...","Siana Capital Management, Info Edge",340000,N/A,None
5,qZense,2019,Bangalore,AgriTech,qZense Labs is building the next-generation Io...,"Rubal Chib, Dr Srishti Batra","Venture Catalysts, 9Unicorns Accelerator Fund",600000,Seed,None
6,MyClassboard,2008,Hyderabad,EdTech,MyClassboard is a full-fledged School / Colleg...,Ajay Sakhamuri,ICICI Bank.,600000,Pre-series A,None
7,Metvy,2018,Gurgaon,Networking platform,AI driven networking platform for individuals ...,Shawrya Mehrotra,HostelFund,0,Pre-series,None
8,Rupeek,2015,Bangalore,FinTech,Rupeek is an online lending platform that spec...,"Amar Prabhu, Ashwin Soni, Sumit Maniyar","KB Investment, Bertelsmann India Investments",45000000,Series C,None
9,Gig India,2017,Pune,Crowdsourcing,GigIndia is a marketplace that provides on-dem...,"Aditya Shirole, Sahil Sharma","Shantanu Deshpande, Subramaniam Ramadorai",1000000,Pre-series A,None


In [615]:
#dropping column 10
final_2020=cleandata_2020.drop('column10', axis=1)


In [616]:
final_2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1055 entries, 0 to 1054
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Company_Brand  1055 non-null   object
 1   Founded        1055 non-null   int32 
 2   HeadQuarter    1055 non-null   object
 3   Sector         1055 non-null   object
 4   What_it_does   1055 non-null   object
 5   Founders       1055 non-null   object
 6   Investor       1055 non-null   object
 7   Amount         1055 non-null   int32 
 8   Stage          1055 non-null   object
dtypes: int32(2), object(7)
memory usage: 66.1+ KB


In [617]:
query_2021 = "SELECT*FROM dbo.LP1_startup_funding2021"
data2021 = pd.read_sql(query_2021, connection)
data2021.head()

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage
0,Unbox Robotics,2019.0,Bangalore,AI startup,Unbox Robotics builds on-demand AI-driven ware...,"Pramod Ghadge, Shahid Memon","BEENEXT, Entrepreneur First","$1,200,000",Pre-series A
1,upGrad,2015.0,Mumbai,EdTech,UpGrad is an online higher education platform.,"Mayank Kumar, Phalgun Kompalli, Ravijot Chugh,...","Unilazer Ventures, IIFL Asset Management","$120,000,000",None
2,Lead School,2012.0,Mumbai,EdTech,LEAD School offers technology based school tra...,"Smita Deorah, Sumeet Mehta","GSV Ventures, Westbridge Capital","$30,000,000",Series D
3,Bizongo,2015.0,Mumbai,B2B E-commerce,Bizongo is a business-to-business online marke...,"Aniket Deb, Ankit Tomar, Sachin Agrawal","CDC Group, IDG Capital","$51,000,000",Series C
4,FypMoney,2021.0,Gurugram,FinTech,"FypMoney is Digital NEO Bank for Teenagers, em...",Kapil Banwari,"Liberatha Kallat, Mukesh Yadav, Dinesh Nagpal","$2,000,000",Seed


In [618]:
def swap_non_numeric_with_numeric(df):
    amounts_column = 'Amount'
    stage_column = 'Stage'
    investor_column = 'Investor'
 
    # Function to find non-numeric values in 'Amounts' column
    def find_non_numeric_values():
        non_numeric_dict = {}
        for index, value in df[amounts_column].items():
            try:
                pd.to_numeric(value)
            except ValueError:
                non_numeric_dict[index] = value
        return non_numeric_dict
 
    # Function to find numeric values in 'Stage' column
    def find_numeric_values():
        numeric_dict = {}
        for index, value in df[stage_column].items():
            try:
                numeric_dict[index] = pd.to_numeric(value)
            except ValueError:
                pass  # Ignore non-numeric values
        return numeric_dict
 
    # Find non-numeric values in 'Amounts' column
    non_numeric_values = find_non_numeric_values()
 
    # Find numeric values in 'Stage' column
    numeric_values = find_numeric_values()
 
    # Swap non-numeric values in 'Amounts' with numeric values in 'Stage'
    for index, non_numeric_value in non_numeric_values.items():
        if index in numeric_values:
            df.at[index, amounts_column] = numeric_values[index]
 
    # Swap numeric values in 'Stage' with non-numeric values in 'Amounts'
    for index, numeric_value in numeric_values.items():
        if index in non_numeric_values:
            df.at[index, stage_column] = non_numeric_values[index]
 
    # Function to find numeric values in 'Investor' column
    def find_numeric_values_founders():
        numeric_dict = {}
        for index, value in df[investor_column].items():
            try:
                numeric_dict[index] = pd.to_numeric(value)
            except ValueError:
                pass  # Ignore non-numeric values
        return numeric_dict
 
    # Function to find non-numeric values in 'Amounts' column
    def find_non_numeric_values_founders():
        non_numeric_dict = {}
        for index, value in df[amounts_column].items():
            try:
                pd.to_numeric(value)
            except ValueError:
                non_numeric_dict[index] = value
        return non_numeric_dict
 
    # Find non-numeric values in 'Amounts' column
    non_numeric_values = find_non_numeric_values_founders()
 
    # Find numeric values in 'Founders' column
    numeric_values = find_numeric_values_founders()
 
    # Swap non-numeric values in 'Amounts' with numeric values in 'Founders'
    for index, non_numeric_value in non_numeric_values.items():
        if index in numeric_values:
            df.at[index, amounts_column] = numeric_values[index]
 
    # Swap numeric values in 'Founders' with non-numeric values in 'Amounts'
    for index, numeric_value in numeric_values.items():
        if index in non_numeric_values:
            df.at[index, investor_column] = non_numeric_values[index]
 
    return df
 
 
def strip_dollar_and_comma(df):
    # Assuming column names are 'Amounts' and 'Stage'
    numeric_columns = ['Investor', 'Amount', 'Stage']
 
    # Iterate through numeric columns and strip dollar sign ($) and comma (,)
    for column in numeric_columns:
        df[column] = df[column].replace('[\$ ,]', '', regex=True)
 
    return df
 
 
def replace_non_numeric_with_zero(df):
    # Assuming column name is 'Amounts'
    amounts_column = 'Amount'
    # Replace non-numeric values in 'Amounts' column with 0
    df[amounts_column] = pd.to_numeric(df[amounts_column], errors='coerce').fillna(0).astype(int)
 
    return df


def fill_null_values(df):
    # Specify the columns to fill with 'N/A'
    columns_to_fill_na = ['HeadQuarter', 'Investor', 'Stage','Sector']
 
    # Fill null values in specified columns with 'N/A'
    df[columns_to_fill_na] = df[columns_to_fill_na].fillna('N/A')

    # Fill null values in 'Founded' with 0   
    df['Founded'] = df['Founded'].fillna(0)

    # Convert 'Founded' values to integers    
    df['Founded'] = df['Founded'].astype(int)
 
    return df


  
 
# Example usage
#data = {'Founders': ['$1,200', '$1,300,000', '', 'D', 'E', 'F', '', 'H', 'J', 'K'],
       # 'Amounts': ['A', 'Undisclosed', '$4,000', '$5,000', 'undisclosed', '$7,000,000', 'Seed', '$4,000', 'series A', '$5,000'],
       # 'Stage': ['Upsparks', 'Series C', 'series c', 'seed', 'seed', 'seed', '$2,000', 'seed', '$3,000', 'series A']}
df = pd.DataFrame(data2021)
 
# Display the DataFrame before processing
print("DataFrame before processing:")
print(df)
info = df.info()
#print("\n Info before processing")
#print(info)
 
# Call the function to replace NaN values in 'Founders' with "N/A"
df['Founders'] = df['Founders'].fillna('N/A')
 
# Call the function to strip dollar sign and comma from numeric values
df = strip_dollar_and_comma(df)
 
# Display the DataFrame after stripping dollar sign and comma
#print("\nDataFrame after stripping dollar sign and comma:")
#print(df)
 
# Call the function to swap non-numeric with numeric values
df = swap_non_numeric_with_numeric(df)
 
# Display the DataFrame after swapping Numerical values in Stage with non-numericals in amounts
#print("\nDataFrame after swapping numeric in 'Stage' with non-numeric values in 'Amount':")
#print(df)
 
# Call the function to swap numeric with non-numeric values in 'Founders'
df = swap_non_numeric_with_numeric(df)
 
# Display the DataFrame after swapping numeric with non-numeric values in 'Founders'
#print("\nDataFrame after swapping numeric in 'Founders' with non-numeric values in 'Amount':")
#print(df)
# Call the function to fill null values with 'N/A'
df = fill_null_values(df)
 # Display the DataFrame after filling null values
print("\nDataFrame after filling null values with 'N/A':")

print(df)
# Call the function to replace non-numeric values in 'Amounts' with 0
df = replace_non_numeric_with_zero(df)

# Call the function to fill null values in  'Founded' with 0
df = fill_null_values(df) 
# Display the DataFrame after filling null values
#print("\nDataFrame after filling null values with 'N/A' and 'Founded' with 0:")
#print(df)

 
 
# Display the DataFrame after replacing non-numeric values with 0
#My amounts column should have all numerics
#print("\nDataFrame after replacing non-numeric values with 0 :")
print("\nDataFrame after processing:")
cleandata_2021=df
cleandata_2021.head(10)

 


DataFrame before processing:
       Company_Brand  Founded HeadQuarter                 Sector  \
0     Unbox Robotics   2019.0   Bangalore             AI startup   
1             upGrad   2015.0      Mumbai                 EdTech   
2        Lead School   2012.0      Mumbai                 EdTech   
3            Bizongo   2015.0      Mumbai         B2B E-commerce   
4           FypMoney   2021.0    Gurugram                FinTech   
...              ...      ...         ...                    ...   
1204        Gigforce   2019.0    Gurugram  Staffing & Recruiting   
1205          Vahdam   2015.0   New Delhi       Food & Beverages   
1206    Leap Finance   2019.0   Bangalore     Financial Services   
1207    CollegeDekho   2015.0    Gurugram                 EdTech   
1208          WeRize   2019.0   Bangalore     Financial Services   

                                           What_it_does  \
0     Unbox Robotics builds on-demand AI-driven ware...   
1        UpGrad is an online higher 

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage
0,Unbox Robotics,2019,Bangalore,AI startup,Unbox Robotics builds on-demand AI-driven ware...,"Pramod Ghadge, Shahid Memon",BEENEXTEntrepreneurFirst,1200000,Pre-seriesA
1,upGrad,2015,Mumbai,EdTech,UpGrad is an online higher education platform.,"Mayank Kumar, Phalgun Kompalli, Ravijot Chugh,...",UnilazerVenturesIIFLAssetManagement,120000000,N/A
2,Lead School,2012,Mumbai,EdTech,LEAD School offers technology based school tra...,"Smita Deorah, Sumeet Mehta",GSVVenturesWestbridgeCapital,30000000,SeriesD
3,Bizongo,2015,Mumbai,B2B E-commerce,Bizongo is a business-to-business online marke...,"Aniket Deb, Ankit Tomar, Sachin Agrawal",CDCGroupIDGCapital,51000000,SeriesC
4,FypMoney,2021,Gurugram,FinTech,"FypMoney is Digital NEO Bank for Teenagers, em...",Kapil Banwari,LiberathaKallatMukeshYadavDineshNagpal,2000000,Seed
5,Urban Company,2014,New Delhi,Home services,Urban Company (Formerly UrbanClap) is a home a...,"Abhiraj Singh Bhal, Raghav Chandra, Varun Khaitan",VyCapital,188000000,N/A
6,Comofi Medtech,2018,Bangalore,HealthTech,Comofi Medtech is a healthcare robotics startup.,Gururaj KB,CIIE.COKIIT-TBI,200000,N/A
7,Qube Health,2016,Mumbai,HealthTech,India's Most Respected Workplace Healthcare Ma...,Gagan Kapur,InflectionPointVentures,0,Pre-seriesA
8,Vitra.ai,2020,Bangalore,Tech Startup,Vitra.ai is an AI-based video translation plat...,Akash Nidhi PS,InflexorVentures,0,Undisclosed
9,Taikee,2010,Mumbai,E-commerce,"Taikee is the ISO-certified, B2B e-commerce pl...","Nidhi Ramachandran, Sachin Chhabra",N/A,1000000,N/A


In [619]:
data2019 = pd.read_csv('C:/Users/KEMUNTO/Downloads/startup_funding2019.csv')
data2019.head()

,Company/Brand,Founded,HeadQuarter,Sector,What it does,Founders,Investor,Amount($),Stage
0,Bombay Shaving,NaN,NaN,Ecommerce,Provides a range of male grooming products,Shantanu Deshpande,Sixth Sense Ventures,"$6,300,000",NaN
1,Ruangguru,2014.0,Mumbai,Edtech,A learning platform that provides topic-based ...,"Adamas Belva Syah Devara, Iman Usman.",General Atlantic,"$150,000,000",Series C
2,Eduisfun,NaN,Mumbai,Edtech,It aims to make learning fun via games.,Jatin Solanki,"Deepak Parekh, Amitabh Bachchan, Piyush Pandey","$28,000,000",Fresh funding
3,HomeLane,2014.0,Chennai,Interior design,Provides interior designing solutions,"Srikanth Iyer, Rama Harinath","Evolvence India Fund (EIF), Pidilite Group, FJ...","$30,000,000",Series D
4,Nu Genes,2004.0,Telangana,AgriTech,"It is a seed company engaged in production, pr...",Narayana Reddy Punyala,Innovation in Food and Agriculture (IFA),"$6,000,000",NaN


In [620]:
# Rename the 'Amount($)' to 'Amount'
data2019.rename(columns={'Amount($)': 'Amount','Company/Brand':'Company_Brand','What it does': 'What_it_does'}, inplace=True)

In [621]:
def swap_non_numeric_with_numeric(df):
    amounts_column = 'Amount'
    stage_column = 'Stage'
    investor_column = 'Investor'
 
    # Function to find non-numeric values in 'Amounts' column
    def find_non_numeric_values():
        non_numeric_dict = {}
        for index, value in df[amounts_column].items():
            try:
                pd.to_numeric(value)
            except ValueError:
                non_numeric_dict[index] = value
        return non_numeric_dict
 
    # Function to find numeric values in 'Stage' column
    def find_numeric_values():
        numeric_dict = {}
        for index, value in df[stage_column].items():
            try:
                numeric_dict[index] = pd.to_numeric(value)
            except ValueError:
                pass  # Ignore non-numeric values
        return numeric_dict
 
    # Find non-numeric values in 'Amounts' column
    non_numeric_values = find_non_numeric_values()
 
    # Find numeric values in 'Stage' column
    numeric_values = find_numeric_values()
 
    # Swap non-numeric values in 'Amounts' with numeric values in 'Stage'
    for index, non_numeric_value in non_numeric_values.items():
        if index in numeric_values:
            df.at[index, amounts_column] = numeric_values[index]
 
    # Swap numeric values in 'Stage' with non-numeric values in 'Amounts'
    for index, numeric_value in numeric_values.items():
        if index in non_numeric_values:
            df.at[index, stage_column] = non_numeric_values[index]
 
    # Function to find numeric values in 'Investor' column
    def find_numeric_values_founders():
        numeric_dict = {}
        for index, value in df[investor_column].items():
            try:
                numeric_dict[index] = pd.to_numeric(value)
            except ValueError:
                pass  # Ignore non-numeric values
        return numeric_dict
 
    # Function to find non-numeric values in 'Amounts' column
    def find_non_numeric_values_founders():
        non_numeric_dict = {}
        for index, value in df[amounts_column].items():
            try:
                pd.to_numeric(value)
            except ValueError:
                non_numeric_dict[index] = value
        return non_numeric_dict
 
    # Find non-numeric values in 'Amounts' column
    non_numeric_values = find_non_numeric_values_founders()
 
    # Find numeric values in 'Founders' column
    numeric_values = find_numeric_values_founders()
 
    # Swap non-numeric values in 'Amounts' with numeric values in 'Founders'
    for index, non_numeric_value in non_numeric_values.items():
        if index in numeric_values:
            df.at[index, amounts_column] = numeric_values[index]
 
    # Swap numeric values in 'Founders' with non-numeric values in 'Amounts'
    for index, numeric_value in numeric_values.items():
        if index in non_numeric_values:
            df.at[index, investor_column] = non_numeric_values[index]
 
    return df
 
 
def strip_dollar_and_comma(df):
    # Assuming column names are 'Amounts' and 'Stage'
    numeric_columns = ['Investor', 'Amount', 'Stage']
 
    # Iterate through numeric columns and strip dollar sign ($) and comma (,)
    for column in numeric_columns:
        df[column] = df[column].replace('[\$ ,]', '', regex=True)
 
    return df
 
 
def replace_non_numeric_with_zero(df):
    # Assuming column name is 'Amounts'
    amounts_column = 'Amount'
    # Replace non-numeric values in 'Amounts' column with 0
    df[amounts_column] = pd.to_numeric(df[amounts_column], errors='coerce').fillna(0).astype(int)
 
    return df


def fill_null_values(df):
    # Specify the columns to fill with 'N/A'
    columns_to_fill_na = ['HeadQuarter', 'Investor', 'Stage','Sector']
 
    # Fill null values in specified columns with 'N/A'
    df[columns_to_fill_na] = df[columns_to_fill_na].fillna('N/A')

    # Fill null values in 'Founded' with 0   
    df['Founded'] = df['Founded'].fillna(0)

    # Convert 'Founded' values to integers    
    df['Founded'] = df['Founded'].astype(int)
 
    return df


  
 
# Example usage
#data = {'Founders': ['$1,200', '$1,300,000', '', 'D', 'E', 'F', '', 'H', 'J', 'K'],
       # 'Amounts': ['A', 'Undisclosed', '$4,000', '$5,000', 'undisclosed', '$7,000,000', 'Seed', '$4,000', 'series A', '$5,000'],
       # 'Stage': ['Upsparks', 'Series C', 'series c', 'seed', 'seed', 'seed', '$2,000', 'seed', '$3,000', 'series A']}
df = pd.DataFrame(data2019)
 
# Display the DataFrame before processing
print("DataFrame before processing:")
print(df)
info = df.info()
#print("\n Info before processing")
#print(info)
 
# Call the function to replace NaN values in 'Founders' with "N/A"
df['Founders'] = df['Founders'].fillna('N/A')
 
# Call the function to strip dollar sign and comma from numeric values
df = strip_dollar_and_comma(df)
 
# Display the DataFrame after stripping dollar sign and comma
#print("\nDataFrame after stripping dollar sign and comma:")
#print(df)
 
# Call the function to swap non-numeric with numeric values
df = swap_non_numeric_with_numeric(df)
 
# Display the DataFrame after swapping Numerical values in Stage with non-numericals in amounts
#print("\nDataFrame after swapping numeric in 'Stage' with non-numeric values in 'Amount':")
#print(df)
 
# Call the function to swap numeric with non-numeric values in 'Founders'
df = swap_non_numeric_with_numeric(df)
 
# Display the DataFrame after swapping numeric with non-numeric values in 'Founders'
#print("\nDataFrame after swapping numeric in 'Founders' with non-numeric values in 'Amount':")
#print(df)
# Call the function to fill null values with 'N/A'
df = fill_null_values(df)
 # Display the DataFrame after filling null values
print("\nDataFrame after filling null values with 'N/A':")

print(df)
# Call the function to replace non-numeric values in 'Amounts' with 0
df = replace_non_numeric_with_zero(df)

# Call the function to fill null values in  'Founded' with 0
df = fill_null_values(df) 
# Display the DataFrame after filling null values
#print("\nDataFrame after filling null values with 'N/A' and 'Founded' with 0:")
#print(df)

 
 
# Display the DataFrame after replacing non-numeric values with 0
#My amounts column should have all numerics
#print("\nDataFrame after replacing non-numeric values with 0 :")
print("\nDataFrame after processing:")
cleandata_2019=df
cleandata_2019.head(10)

 


DataFrame before processing:
      Company_Brand  Founded HeadQuarter                   Sector  \
0    Bombay Shaving      NaN         NaN                Ecommerce   
1         Ruangguru   2014.0      Mumbai                   Edtech   
2          Eduisfun      NaN      Mumbai                   Edtech   
3          HomeLane   2014.0     Chennai          Interior design   
4          Nu Genes   2004.0   Telangana                 AgriTech   
..              ...      ...         ...                      ...   
84     Infra.Market      NaN      Mumbai                Infratech   
85              Oyo   2013.0    Gurugram              Hospitality   
86       GoMechanic   2016.0       Delhi  Automobile & Technology   
87           Spinny   2015.0       Delhi               Automobile   
88  Ess Kay Fincorp      NaN   Rajasthan                  Banking   

                                         What_it_does  \
0          Provides a range of male grooming products   
1   A learning platform that

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage
0,Bombay Shaving,0,N/A,Ecommerce,Provides a range of male grooming products,Shantanu Deshpande,SixthSenseVentures,6300000,N/A
1,Ruangguru,2014,Mumbai,Edtech,A learning platform that provides topic-based ...,"Adamas Belva Syah Devara, Iman Usman.",GeneralAtlantic,150000000,SeriesC
2,Eduisfun,0,Mumbai,Edtech,It aims to make learning fun via games.,Jatin Solanki,DeepakParekhAmitabhBachchanPiyushPandey,28000000,Freshfunding
3,HomeLane,2014,Chennai,Interior design,Provides interior designing solutions,"Srikanth Iyer, Rama Harinath",EvolvenceIndiaFund(EIF)PidiliteGroupFJLabs,30000000,SeriesD
4,Nu Genes,2004,Telangana,AgriTech,"It is a seed company engaged in production, pr...",Narayana Reddy Punyala,InnovationinFoodandAgriculture(IFA),6000000,N/A
5,FlytBase,0,Pune,Technology,A drone automation platform,Nitin Gupta,Undisclosed,0,Undisclosed
6,Finly,0,Bangalore,SaaS,It builds software products that makes work si...,"Vivek AG, Veekshith C Rai",SocialCapitalAngelListIndiaGembaCapitalOmphalo...,0,Undisclosed
7,Kratikal,2013,Noida,Technology,It is a product-based cybersecurity solutions ...,"Pavan Kushwaha, Paratosh Bansal, Dip Jung Thapa",GildaVCArtVentureRajeevChitrabhanu.,1000000,PreseriesA
8,Quantiphi,0,N/A,AI & Tech,It is an AI and big data services company prov...,Renuka Ramnath,MultiplesAlternateAssetManagement,20000000,SeriesA
9,Lenskart,2010,Delhi,E-commerce,It is a eyewear company,"Peyush Bansal, Amit Chaudhary, Sumeet Kapahi",SoftBank,275000000,SeriesG


In [622]:
url = 'https://raw.github.com/Azubi-Africa/Career_Accelerator_LP1-Data_Analysis/main/startup_funding2018.csv'
download = requests.get(url).content
data2018 = pd.read_csv(io.StringIO(download.decode('utf-8')))
data2018.head()

,Company Name,Industry,Round/Series,Amount,Location,About Company
0,TheCollegeFever,"Brand Marketing, Event Promotion, Marketing, S...",Seed,250000,"Bangalore, Karnataka, India","TheCollegeFever is a hub for fun, fiesta and f..."
1,Happy Cow Dairy,"Agriculture, Farming",Seed,"₹40,000,000","Mumbai, Maharashtra, India",A startup which aggregates milk from dairy far...
2,MyLoanCare,"Credit, Financial Services, Lending, Marketplace",Series A,"₹65,000,000","Gurgaon, Haryana, India",Leading Online Loans Marketplace in India
3,PayMe India,"Financial Services, FinTech",Angel,2000000,"Noida, Uttar Pradesh, India",PayMe India is an innovative FinTech organizat...
4,Eunimart,"E-Commerce Platforms, Retail, SaaS",Seed,—,"Hyderabad, Andhra Pradesh, India",Eunimart is a one stop solution for merchants ...


In [623]:
new_names = {'Company Name':'Company_Brand','Industry':'Sector','Round/Series':'Stage','Location':'HeadQuarter','About Company':'What_it_does'}
df = data2018.rename(columns=new_names)
df.head()

,Company_Brand,Sector,Stage,Amount,HeadQuarter,What_it_does
0,TheCollegeFever,"Brand Marketing, Event Promotion, Marketing, S...",Seed,250000,"Bangalore, Karnataka, India","TheCollegeFever is a hub for fun, fiesta and f..."
1,Happy Cow Dairy,"Agriculture, Farming",Seed,"₹40,000,000","Mumbai, Maharashtra, India",A startup which aggregates milk from dairy far...
2,MyLoanCare,"Credit, Financial Services, Lending, Marketplace",Series A,"₹65,000,000","Gurgaon, Haryana, India",Leading Online Loans Marketplace in India
3,PayMe India,"Financial Services, FinTech",Angel,2000000,"Noida, Uttar Pradesh, India",PayMe India is an innovative FinTech organizat...
4,Eunimart,"E-Commerce Platforms, Retail, SaaS",Seed,—,"Hyderabad, Andhra Pradesh, India",Eunimart is a one stop solution for merchants ...


In [624]:
import numpy as np
# Add new columns with NaN values

df['Founded'] = '0'
df['Founders'] = 'N/A'
df['Investor'] = 'N/A'

In [625]:
# Rearrange columns
columns_order = ['Company_Brand','Founded', 'HeadQuarter', 'Sector','What_it_does','Founders','Investor','Amount','Stage']

df = df.reindex(columns=columns_order)

data2018 = df
data2018

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage
0,TheCollegeFever,0,"Bangalore, Karnataka, India","Brand Marketing, Event Promotion, Marketing, S...","TheCollegeFever is a hub for fun, fiesta and f...",N/A,N/A,250000,Seed
1,Happy Cow Dairy,0,"Mumbai, Maharashtra, India","Agriculture, Farming",A startup which aggregates milk from dairy far...,N/A,N/A,"₹40,000,000",Seed
2,MyLoanCare,0,"Gurgaon, Haryana, India","Credit, Financial Services, Lending, Marketplace",Leading Online Loans Marketplace in India,N/A,N/A,"₹65,000,000",Series A
3,PayMe India,0,"Noida, Uttar Pradesh, India","Financial Services, FinTech",PayMe India is an innovative FinTech organizat...,N/A,N/A,2000000,Angel
4,Eunimart,0,"Hyderabad, Andhra Pradesh, India","E-Commerce Platforms, Retail, SaaS",Eunimart is a one stop solution for merchants ...,N/A,N/A,—,Seed
...,...,...,...,...,...,...,...,...,...
521,Udaan,0,"Bangalore, Karnataka, India","B2B, Business Development, Internet, Marketplace","Udaan is a B2B trade platform, designed specif...",N/A,N/A,225000000,Series C
522,Happyeasygo Group,0,"Haryana, Haryana, India","Tourism, Travel",HappyEasyGo is an online travel domain.,N/A,N/A,—,Series A
523,Mombay,0,"Mumbai, Maharashtra, India","Food and Beverage, Food Delivery, Internet",Mombay is a unique opportunity for housewives ...,N/A,N/A,7500,Seed
524,Droni Tech,0,"Mumbai, Maharashtra, India",Information Technology,Droni Tech manufacture UAVs and develop softwa...,N/A,N/A,"₹35,000,000",Seed


In [626]:
data2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 526 entries, 0 to 525
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Company_Brand  526 non-null    object
 1   Founded        526 non-null    object
 2   HeadQuarter    526 non-null    object
 3   Sector         526 non-null    object
 4   What_it_does   526 non-null    object
 5   Founders       526 non-null    object
 6   Investor       526 non-null    object
 7   Amount         526 non-null    object
 8   Stage          526 non-null    object
dtypes: object(9)
memory usage: 37.1+ KB


In [627]:
#cleaning the whole database after rearranging the columns ,inserting the columns and filling the inserted columns with n/a and 0

In [628]:
def swap_non_numeric_with_numeric(df):
    amounts_column = 'Amount'
    stage_column = 'Stage'
    investor_column = 'Investor'
 
    # Function to find non-numeric values in 'Amounts' column
    def find_non_numeric_values():
        non_numeric_dict = {}
        for index, value in df[amounts_column].items():
            try:
                pd.to_numeric(value)
            except ValueError:
                non_numeric_dict[index] = value
        return non_numeric_dict
 
    # Function to find numeric values in 'Stage' column
    def find_numeric_values():
        numeric_dict = {}
        for index, value in df[stage_column].items():
            try:
                numeric_dict[index] = pd.to_numeric(value)
            except ValueError:
                pass  # Ignore non-numeric values
        return numeric_dict
 
    # Find non-numeric values in 'Amounts' column
    non_numeric_values = find_non_numeric_values()
 
    # Find numeric values in 'Stage' column
    numeric_values = find_numeric_values()
 
    # Swap non-numeric values in 'Amounts' with numeric values in 'Stage'
    for index, non_numeric_value in non_numeric_values.items():
        if index in numeric_values:
            df.at[index, amounts_column] = numeric_values[index]
 
    # Swap numeric values in 'Stage' with non-numeric values in 'Amounts'
    for index, numeric_value in numeric_values.items():
        if index in non_numeric_values:
            df.at[index, stage_column] = non_numeric_values[index]
 
    # Function to find numeric values in 'Investor' column
    def find_numeric_values_founders():
        numeric_dict = {}
        for index, value in df[investor_column].items():
            try:
                numeric_dict[index] = pd.to_numeric(value)
            except ValueError:
                pass  # Ignore non-numeric values
        return numeric_dict
 
    # Function to find non-numeric values in 'Amounts' column
    def find_non_numeric_values_founders():
        non_numeric_dict = {}
        for index, value in df[amounts_column].items():
            try:
                pd.to_numeric(value)
            except ValueError:
                non_numeric_dict[index] = value
        return non_numeric_dict
 
    # Find non-numeric values in 'Amounts' column
    non_numeric_values = find_non_numeric_values_founders()
 
    # Find numeric values in 'Founders' column
    numeric_values = find_numeric_values_founders()
 
    # Swap non-numeric values in 'Amounts' with numeric values in 'Founders'
    for index, non_numeric_value in non_numeric_values.items():
        if index in numeric_values:
            df.at[index, amounts_column] = numeric_values[index]
 
    # Swap numeric values in 'Founders' with non-numeric values in 'Amounts'
    for index, numeric_value in numeric_values.items():
        if index in non_numeric_values:
            df.at[index, investor_column] = non_numeric_values[index]
 
    return df
 
 
def strip_dollar_and_comma(df):
    # Assuming column names are 'Amounts' and 'Stage'
    numeric_columns = ['Investor', 'Amount', 'Stage']
 
    # Iterate through numeric columns and strip dollar sign ($) and comma (,)
    for column in numeric_columns:
        df[column] = df[column].replace('[\₹ ,]', '', regex=True)
 
    return df
 
 
def replace_non_numeric_with_zero(df):
    # Assuming column name is 'Amounts'
    amounts_column = 'Amount'
    # Replace non-numeric values in 'Amounts' column with 0
    df[amounts_column] = pd.to_numeric(df[amounts_column], errors='coerce').fillna(0).astype(int)
 
    return df


def fill_null_values(df):
    # Specify the columns to fill with 'N/A'
    columns_to_fill_na = ['HeadQuarter', 'Investor', 'Stage','Sector']
 
    # Fill null values in specified columns with 'N/A'
    df[columns_to_fill_na] = df[columns_to_fill_na].fillna('N/A')

    # Fill null values in 'Founded' with 0   
    df['Founded'] = df['Founded'].fillna(0)

    # Convert 'Founded' values to integers    
    df['Founded'] = df['Founded'].astype(int)
 
    return df


  
 
# Example usage
#data = {'Founders': ['$1,200', '$1,300,000', '', 'D', 'E', 'F', '', 'H', 'J', 'K'],
       # 'Amounts': ['A', 'Undisclosed', '$4,000', '$5,000', 'undisclosed', '$7,000,000', 'Seed', '$4,000', 'series A', '$5,000'],
       # 'Stage': ['Upsparks', 'Series C', 'series c', 'seed', 'seed', 'seed', '$2,000', 'seed', '$3,000', 'series A']}
df = pd.DataFrame(data2018)
 
# Display the DataFrame before processing
print("DataFrame before processing:")
print(df)
info = df.info()
#print("\n Info before processing")
#print(info)
 
# Call the function to replace NaN values in 'Founders' with "N/A"
df['Founders'] = df['Founders'].fillna('N/A')
 
# Call the function to strip dollar sign and comma from numeric values
df = strip_dollar_and_comma(df)
 
# Display the DataFrame after stripping dollar sign and comma
print("\nDataFrame after stripping dollar sign and comma:")
print(df)
 
# Call the function to swap non-numeric with numeric values
df = swap_non_numeric_with_numeric(df)
 
# Display the DataFrame after swapping Numerical values in Stage with non-numericals in amounts
print("\nDataFrame after swapping numeric in 'Stage' with non-numeric values in 'Amount':")
print(df)
 
# Call the function to swap numeric with non-numeric values in 'Founders'
df = swap_non_numeric_with_numeric(df)
 
# Display the DataFrame after swapping numeric with non-numeric values in 'Founders'
#print("\nDataFrame after swapping numeric in 'Founders' with non-numeric values in 'Amount':")
#print(df)
# Call the function to fill null values with 'N/A'
df = fill_null_values(df)
 # Display the DataFrame after filling null values
print("\nDataFrame after filling null values with 'N/A':")

print(df)
# Call the function to replace non-numeric values in 'Amounts' with 0
df = replace_non_numeric_with_zero(df)

# Call the function to fill null values in  'Founded' with 0
df = fill_null_values(df) 
# Display the DataFrame after filling null values
#print("\nDataFrame after filling null values with 'N/A' and 'Founded' with 0:")
#print(df)

 
 
# Display the DataFrame after replacing non-numeric values with 0
#My amounts column should have all numerics
#print("\nDataFrame after replacing non-numeric values with 0 :")
print("\nDataFrame after processing:")
cleandata_2018=df
cleandata_2018.head(10)

 


DataFrame before processing:
         Company_Brand Founded                       HeadQuarter  \
0      TheCollegeFever       0       Bangalore, Karnataka, India   
1      Happy Cow Dairy       0        Mumbai, Maharashtra, India   
2           MyLoanCare       0           Gurgaon, Haryana, India   
3          PayMe India       0       Noida, Uttar Pradesh, India   
4             Eunimart       0  Hyderabad, Andhra Pradesh, India   
..                 ...     ...                               ...   
521              Udaan       0       Bangalore, Karnataka, India   
522  Happyeasygo Group       0           Haryana, Haryana, India   
523             Mombay       0        Mumbai, Maharashtra, India   
524         Droni Tech       0        Mumbai, Maharashtra, India   
525            Netmeds       0        Chennai, Tamil Nadu, India   

                                                Sector  \
0    Brand Marketing, Event Promotion, Marketing, S...   
1                                 Agri


DataFrame after swapping numeric in 'Stage' with non-numeric values in 'Amount':
         Company_Brand Founded                       HeadQuarter  \
0      TheCollegeFever       0       Bangalore, Karnataka, India   
1      Happy Cow Dairy       0        Mumbai, Maharashtra, India   
2           MyLoanCare       0           Gurgaon, Haryana, India   
3          PayMe India       0       Noida, Uttar Pradesh, India   
4             Eunimart       0  Hyderabad, Andhra Pradesh, India   
..                 ...     ...                               ...   
521              Udaan       0       Bangalore, Karnataka, India   
522  Happyeasygo Group       0           Haryana, Haryana, India   
523             Mombay       0        Mumbai, Maharashtra, India   
524         Droni Tech       0        Mumbai, Maharashtra, India   
525            Netmeds       0        Chennai, Tamil Nadu, India   

                                                Sector  \
0    Brand Marketing, Event Promotion, Mark

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage
0,TheCollegeFever,0,"Bangalore, Karnataka, India","Brand Marketing, Event Promotion, Marketing, S...","TheCollegeFever is a hub for fun, fiesta and f...",N/A,N/A,250000,Seed
1,Happy Cow Dairy,0,"Mumbai, Maharashtra, India","Agriculture, Farming",A startup which aggregates milk from dairy far...,N/A,N/A,40000000,Seed
2,MyLoanCare,0,"Gurgaon, Haryana, India","Credit, Financial Services, Lending, Marketplace",Leading Online Loans Marketplace in India,N/A,N/A,65000000,SeriesA
3,PayMe India,0,"Noida, Uttar Pradesh, India","Financial Services, FinTech",PayMe India is an innovative FinTech organizat...,N/A,N/A,2000000,Angel
4,Eunimart,0,"Hyderabad, Andhra Pradesh, India","E-Commerce Platforms, Retail, SaaS",Eunimart is a one stop solution for merchants ...,N/A,N/A,0,Seed
5,Hasura,0,"Bengaluru, Karnataka, India","Cloud Infrastructure, PaaS, SaaS",Hasura is a platform that allows developers to...,N/A,N/A,1600000,Seed
6,Tripshelf,0,"Kalkaji, Delhi, India","Internet, Leisure, Marketplace",Tripshelf is an online market place for holida...,N/A,N/A,16000000,Seed
7,Hyperdata.IO,0,"Hyderabad, Andhra Pradesh, India",Market Research,Hyperdata combines advanced machine learning w...,N/A,N/A,50000000,Angel
8,Freightwalla,0,"Mumbai, Maharashtra, India","Information Services, Information Technology",Freightwalla is an international forwarder tha...,N/A,N/A,0,Seed
9,Microchip Payments,0,"Bangalore, Karnataka, India",Mobile Payments,Microchip payments is a mobile-based payment a...,N/A,N/A,0,Seed


In [629]:
# checking the null values after cleaning 
cleandata_2018.isna().sum()

Company_Brand    0
Founded          0
HeadQuarter      0
Sector           0
What_it_does     0
Founders         0
Investor         0
Amount           0
Stage            0
dtype: int64

In [636]:
# concantinating the databases
# Assuming you have datasets named data2021, data2020, data2019, data2018
 
# Sample data for illustration
#final_2020 = pd.final_2020
#cleandata_2021 = pd.cleandata_2021
#cleandata_2019 = pd.cleandata_2019
#cleandata_2018 = pd.cleandata_2018
 
# Concatenate datasets and create an additional 'Year' column
concatenated_data = pd.concat([final_2020, cleandata_2021, cleandata_2019, cleandata_2018], keys=['2020', '2021', '2019', '2018'], names=['Year'])
full_data = concatenated_data


In [638]:
#checking the database 
full_data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2879 entries, ('2020', 0) to ('2018', 525)
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Company_Brand  2879 non-null   object
 1   Founded        2879 non-null   int32 
 2   HeadQuarter    2879 non-null   object
 3   Sector         2879 non-null   object
 4   What_it_does   2879 non-null   object
 5   Founders       2879 non-null   object
 6   Investor       2879 non-null   object
 7   Amount         2879 non-null   int32 
 8   Stage          2879 non-null   object
dtypes: int32(2), object(7)
memory usage: 230.4+ KB


In [639]:
# finding null values
full_data.isna().sum()

Company_Brand    0
Founded          0
HeadQuarter      0
Sector           0
What_it_does     0
Founders         0
Investor         0
Amount           0
Stage            0
dtype: int64

Exploratory Data Analysis